In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import urllib
from os import path
import pandas as pd
import time
import pickle
driver=webdriver.Chrome()

In [2]:
amount_of_routes = 30
climbing_dic = {'name':{},'grade':{},'completed':{},'location_1':{},'location_2':{}}
error_log=[]

In [9]:
# code will loop through all possible climbs by webpage#
# outputs: name, grade, count of times climbed, regional location, sublocation

def web_scrape(climbing_dic,error_log,amount_of_routes,start_point=1):
    base_location='https://www.ukclimbing.com/logbook/c.php?i='
    for i in range(start_point,amount_of_routes):
        page_location = '{0}{1}'.format(base_location,i)
        driver.get(page_location)
        try:
            el=driver.find_element_by_xpath("//small")
            el1=driver.find_element_by_xpath("//div[@class='progress-bar bg-success polltype1']")
            title=el1.get_property('title')
            climb_name = driver.title
            climb_name = climb_name[climb_name.find('-')+2:]
            climb_grade=el.text
            climb_completed=int(title[title.find('of')+3:title.find('votes')-1])
            location_1=driver.find_element_by_xpath("//li[@class='breadcrumb-item d-none d-sm-inline-block']")
            location_1 = location_1.text
            location_2 = driver.find_elements_by_xpath("//li[@class='breadcrumb-item']")[1].text
            climbing_dic['name'][i]=climb_name
            climbing_dic['grade'][i]=climb_grade
            climbing_dic['completed'][i]=climb_completed
            climbing_dic['location_1'][i]=location_1
            climbing_dic['location_2'][i]=location_2
            climb_style=driver.find_elements_by_xpath("//div[@class='col-4 tiny text-right text-success']")
            climb_style_completed=driver.find_elements_by_xpath("//div[@class='card-body'][1]//div[@class='progress']//div[@class='progress-bar bg-success']")
            # adds all the potential styles that a climb could be done in
            for counter, style in enumerate(climb_style):
                if style.text not in climbing_dic.keys():
                    climbing_dic[style.text]={}
                climbing_dic[style.text][i]=float(climb_style_completed[counter].get_attribute('aria-valuenow'))
        except:
            error_log.append(i)
    # pickle dictionary for later use
    pickle_file = open('climbingdic.pkl','wb')
    pickle.dump(climbing_dic,pickle_file)
    print('errored pages: {0}'.format(len(error_log)))
    print('Records scraped: {0}'.format(len(climbing_dic['name'])))

In [10]:
web_scrape(climbing_dic=climbing_dic,error_log=error_log,amount_of_routes=amount_of_routes,start_point=len(climbing_dic['name']))

errored pages: 1
Records scraped: 29


In [11]:
pd.DataFrame(climbing_dic)

,Alt Leads,Bouldered,Flashed (β),Ground Up,Lead,Onsighted,Redpoint,Repeated,Soloed,completed,grade,location_1,location_2,name
1,61.781076,NaN,1.484230,0.371058,13.172542,45.454545,0.371058,2.597403,0.371058,98,E1 5b,Orkney Islands,Logbooks,'East Face Route (Original Route)'
2,56.250000,NaN,NaN,NaN,25.000000,75.000000,NaN,NaN,NaN,8,E3 5c,Orkney Islands,Logbooks,'A Few Dollars More'
3,8.906883,0.404858,NaN,NaN,7.287449,29.959514,NaN,2.429150,55.465587,31,VD,Arran,Logbooks,'A'Chir Ridge'
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,HVS 5a,Arran,Logbooks,'Minaret'
5,54.054054,NaN,2.702703,NaN,18.918919,54.054054,NaN,NaN,10.810811,12,S,Arran,Logbooks,'Pagoda Ridge'
6,65.000000,NaN,NaN,NaN,15.000000,35.000000,NaN,NaN,NaN,7,VS 4b,Arran,Logbooks,'Mosque'
7,NaN,7.142857,7.142857,NaN,21.428571,71.428571,NaN,14.285714,64.285714,2,E3 5c,North Yorkshire,Logbooks,'Envy of Angels'
8,NaN,5.000000,7.500000,2.500000,42.500000,52.500000,5.000000,5.000000,30.000000,4,HVS 5b,North Yorkshire,Logbooks,'Angelic Upstart'
9,NaN,0.473934,3.317536,0.473934,57.819905,43.601896,0.473934,7.109005,5.213270,20,VS 5a,North Yorkshire,Logbooks,'Pram Pusher's Paradise'
10,NaN,4.761905,4.761905,NaN,28.571429,76.190476,NaN,NaN,42.857143,2,HVS 5b,North Yorkshire,Logbooks,'A Step in the Right Direction'
